In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("humana.csv")

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Preprocessing Dataset
1. Drop Duplicates
2. Filter the data which match the requirement of the compettion
3. Subset the dataset for easier preprocessing
4. Set some important features to calculate the medicine taken, naive periods, LTOT times

In [3]:
df.shape

(5943463, 20)

In [4]:
f = {"Days" : ["min"]}
df1 = df.groupby(["id"]).agg(f).reset_index()
df1.columns = ['id', 'min_day']

In [5]:
df = pd.merge( df, df1, on=['id'], how = 'left')

In [6]:
not_null = pd.notnull(df["PAY_DAY_SUPPLY_CNT"])

In [7]:
filter1 = df[not_null]
opioid1 = filter1.loc[(filter1.event_descr =="RX Claim - Paid") & (filter1.MME>0)]
opioid1 = opioid1.reset_index(drop = True)
opioid1 = opioid1.sort_values(['id','Days',	'PAY_DAY_SUPPLY_CNT'],ascending=[True, True, True])
opioid1 = opioid1.reset_index(drop = True)

In [8]:
opioid = opioid1.loc[:,['id', 'PAY_DAY_SUPPLY_CNT','MME', 'Days', 'min_day']]
opioid.shape

(240818, 5)

In [9]:
opioid = opioid.loc[opioid.Days<0]
opioid.shape

(75892, 5)

In [10]:
opioid = opioid.reset_index(drop=True)

# Create new features

In [11]:
#The number of Period
opioid.loc[:,'Period'] = np.nan

#The end days of period# time
opioid.loc[:,'max_p_end'] = np.nan

#How many medical the patient buy
opioid.loc[:,'med_buy'] = np.nan

#How many days the patient take all of drug
opioid.loc[:,'day_count'] = np.nan

#Is the patient can be attribute to navie
opioid.loc[:,'windows'] =np.nan

#The period of windows
opioid.loc[:,'w_period'] =np.nan

#the raiod of medicane taken in that period
opioid.loc[:,'windows_count'] =np.nan

Before the day 0, all time should be attribute to the first period

In [12]:
opioid.loc[0,'Period'] = 1
opioid.loc[0,'max_p_end'] = -614+180
opioid.loc[0,'med_buy'] = 5
opioid.loc[0,'day_count'] =5

In [13]:
#find the first period, Since the condition may not consider the last rows, the info of last row should be imputted manually
for i in range(1,len(opioid)):
    if opioid.loc[i,'Days'] >= opioid.loc[i,'min_day'] + 90 and opioid.loc[i,'id'] != opioid.loc[i-1,'id'] and opioid.loc[i,'id'] == opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = 1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], opioid.loc[i+1,'Days']-opioid.loc[i,'Days'], 180)
        
    if opioid.loc[i,'Days'] >= opioid.loc[i,'min_day'] + 90 and opioid.loc[i,'id'] != opioid.loc[i-1,'id'] and opioid.loc[i,'id'] != opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = 1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        
    if opioid.loc[i,'Days'] < opioid.loc[i,'min_day'] + 90 and opioid.loc[i,'id'] == opioid.loc[i+1,'id']:
        opioid.loc[i,'day_count'] =0
        opioid.loc[i,'Period'] = "not_Qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], opioid.loc[i+1,'Days']-opioid.loc[i,'Days'])
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'w_period'] = 0
        opioid.loc[i,'windows'] = "not_naive"
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] + opioid.loc[i,'windows_count'] + 90
            
    if opioid.loc[i,'Days'] < opioid.loc[i,'min_day'] + 90 and opioid.loc[i,'id'] != opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = "not_Qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'w_period'] = 0
        opioid.loc[i,'windows'] = "not_naive"
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] + opioid.loc[i,'windows_count'] + 90
        opioid.loc[i,'day_count'] =0
        
    if opioid.loc[i-1,'Period'] == "not_Qualified" and opioid.loc[i,'id'] == opioid.loc[i-1,'id'] and opioid.loc[i,'Days'] >= opioid.loc[i-1,'max_p_end'] and opioid.loc[i,'id'] == opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = 1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], opioid.loc[i+1,'Days']-opioid.loc[i,'Days'], 180)
        
    if opioid.loc[i-1,'Period'] == "not_Qualified" and opioid.loc[i,'id'] == opioid.loc[i-1,'id'] and opioid.loc[i,'Days'] >= opioid.loc[i-1,'max_p_end'] and opioid.loc[i,'id'] != opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = 1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        
    if opioid.loc[i-1,'Period'] == "not_Qualified" and opioid.loc[i,'id'] == opioid.loc[i-1,'id'] and opioid.loc[i,'Days'] < opioid.loc[i-1,'max_p_end'] and opioid.loc[i,'id'] ==opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = "not_Qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], opioid.loc[i+1,'Days']-opioid.loc[i,'Days'])
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'w_period'] = 0
        opioid.loc[i,'windows'] = "not_naive"
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] + opioid.loc[i,'windows_count'] + 90
        opioid.loc[i,'day_count'] =0
        
    if opioid.loc[i-1,'Period'] == "not_Qualified" and opioid.loc[i,'id'] == opioid.loc[i-1,'id'] and opioid.loc[i,'Days'] < opioid.loc[i-1,'max_p_end'] and opioid.loc[i,'id'] !=opioid.loc[i+1,'id']:
        opioid.loc[i,'Period'] = "not_Qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'w_period'] = 0
        opioid.loc[i,'windows'] = "not_naive"
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] + opioid.loc[i,'windows_count'] + 90
        opioid.loc[i,'day_count'] =0

KeyError: 75892

In [14]:
# input last manually
opioid.loc[75891,'max_p_end'] = -164 +30+90
opioid.loc[75891,'med_buy'] = 30
opioid.loc[75891,'day_count'] =0
opioid.loc[75891,'windows'] = "not_naive"
opioid.loc[75891,'w_period'] = 0 
opioid.loc[75891,'windows_count'] = 30
opioid.loc[75891,'Period'] = 'not_Qualified'

In [15]:
opioid.day_count.isna().sum()

10159

In [16]:
test = opioid['med_buy'].index[opioid['med_buy'].apply(np.isnan)]
test

Int64Index([    1,     2,   129,   131,   132,   133,   134,   135,   136,
              137,
            ...
            75667, 75697, 75698, 75699, 75700, 75701, 75702, 75703, 75704,
            75737],
           dtype='int64', length=10159)

In [17]:
# This part can be update to a clear and easy version.
for i in test:
    
    if opioid.loc[i,'id']== opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days'] >= (opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90):
        opioid.loc[i,'Period'] = opioid.loc[i-1,'Period'] +1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days']+180
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], (opioid.loc[i+1,'Days']-opioid.loc[i,'Days']),180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        
        
    #ensure the same iD， the last day is not Naived, in the same period, the interval is less than 90 days
    if opioid.loc[i,'id']== opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days'] < opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90 and opioid.loc[i,'Days'] < opioid.loc[i-1,'max_p_end']:
        opioid.loc[i,'Period'] = opioid.loc[i-1,'Period']
        opioid.loc[i,'max_p_end'] = opioid.loc[i-1,'max_p_end']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], (opioid.loc[i+1,'Days']-opioid.loc[i,'Days']),opioid.loc[i-1,'max_p_end']-opioid.loc[i,'Days'])
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        
    #  ensure the same iD， the last day is not Naived, the interval is less than 90 days, but not larger than 180 days
    if opioid.loc[i,'id']== opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days']< opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90 and opioid.loc[i,'Days'] >= opioid.loc[i-1,'max_p_end']:
        opioid.loc[i,'Period'] = "not_qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], (opioid.loc[i+1,'Days']- opioid.loc[i,'Days']))
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'windows_count'] + opioid.loc[i,'Days'] + 90
        opioid.loc[i,'day_count'] = 0
        opioid.loc[i,'windows'] = "x_naive"
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'w_period'] = opioid.loc[i-1,'Period'] + 1
    
    # ensure the same ID,  is not belong to new period since the difference between this day and naive day is less than 90 days 
    if opioid.loc[i,'id']== opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] == "x_naive" and opioid.loc[i,'Days'] < opioid.loc[i-1,"max_p_end"]:
        opioid.loc[i,'Period'] = "not_qualified"
        opioid.loc[i,'windows_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], (opioid.loc[i+1,'Days']- opioid.loc[i,'Days']))
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'windows_count'] + opioid.loc[i,'Days'] + 90
        opioid.loc[i,'day_count'] = 0
        opioid.loc[i,'windows'] = "x_naive"
        opioid.loc[i,'w_period'] = opioid.loc[i-1,'w_period']
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
    
    # ensure the same ID, the difference to the last naive period is larger than 90days  
    if opioid.loc[i,'id']== opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] == "x_naive" and opioid.loc[i,'Days'] >= opioid.loc[i-1,"max_p_end"]:
        opioid.loc[i,'Period'] = opioid.loc[i-1,'w_period']
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], (opioid.loc[i+1,'Days']-opioid.loc[i,'Days']),180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
    
    # the next row is another ID，前一天是Not Naived, 间隔大于90天
    if opioid.loc[i,'id']!= opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] == "x_naive" and opioid.loc[i,'Days'] >= opioid.loc[i-1,"max_p_end"]:
        opioid.loc[i,'Period'] = opioid.loc[i-1,'w_period']
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] +180
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
 

    #Next part consider the same thing, but the different is the next day may have different ID
    if opioid.loc[i,'id']!= opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] == "x_naive" and opioid.loc[i,'Days'] < opioid.loc[i-1,"max_p_end"]:
        opioid.loc[i,'Period'] = "not_qualified"
        opioid.loc[i,'windows_count'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'windows_count'] + opioid.loc[i,'Days'] + 90
        opioid.loc[i,'day_count'] = 0
        opioid.loc[i,'windows'] = "x_naive"
        opioid.loc[i,'w_period'] = opioid.loc[i-1,'w_period']
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
    
    
    if opioid.loc[i,'id']!= opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days'] >= (opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90):
        opioid.loc[i,'Period'] = opioid.loc[i-1,'Period'] +1
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days']+180
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'], 180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        
   
    if opioid.loc[i,'id']!= opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days'] < opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90 and opioid.loc[i,'Days'] < opioid.loc[i-1,'max_p_end']:
        opioid.loc[i,'Period'] = opioid.loc[i-1,'Period']
        opioid.loc[i,'max_p_end'] = opioid.loc[i-1,'max_p_end']
        opioid.loc[i,'day_count'] = min(opioid.loc[i,'PAY_DAY_SUPPLY_CNT'],180)
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        
   
    if opioid.loc[i,'id']!= opioid.loc[i+1,'id'] and opioid.loc[i-1,'windows'] != "x_naive" and opioid.loc[i,'Days']< opioid.loc[i-1,"Days"] + max(opioid.loc[i-1,"PAY_DAY_SUPPLY_CNT"],opioid.loc[i-1,'day_count'])+90 and opioid.loc[i,'Days'] >= opioid.loc[i-1,'max_p_end']:
        opioid.loc[i,'Period'] ="not_qualified"
        opioid.loc[i,'max_p_end'] = opioid.loc[i,'Days'] + opioid.loc[i,'PAY_DAY_SUPPLY_CNT']+90
        opioid.loc[i,'day_count'] = 0
        opioid.loc[i,'med_buy'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']
        opioid.loc[i,'windows'] = "x_naive"
        opioid.loc[i,'w_period'] = opioid.loc[i-1,'Period']+1
        opioid.loc[i,'windows_count'] = opioid.loc[i,'PAY_DAY_SUPPLY_CNT']

In [ ]:
opioid.to_csv('before_0.csv')